# Proyecto Final - Precios de SUVs 

In [1]:
%load_ext autoreload
%autoreload 2

### Analisis dataset crudo

In [2]:
import pandas as pd
# Cargar el dataset
df_raw = pd.read_csv("dataset/raw/pf_suvs_i302_1s2025.csv")
df_raw = pd.DataFrame(df_raw)

In [3]:
from src.data_exploration import*

In [4]:
exploratory_analysis(df_raw, target_col="Precio")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18254 entries, 0 to 18253
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               18254 non-null  int64  
 1   Marca                    18254 non-null  object 
 2   Modelo                   18254 non-null  object 
 3   Año                      18254 non-null  float64
 4   Versión                  18254 non-null  object 
 5   Color                    17865 non-null  object 
 6   Tipo de combustible      18254 non-null  object 
 7   Puertas                  18254 non-null  float64
 8   Transmisión              18239 non-null  object 
 9   Motor                    18216 non-null  object 
 10  Tipo de carrocería       18254 non-null  object 
 11  Con cámara de retroceso  4691 non-null   object 
 12  Kilómetros               18254 non-null  object 
 13  Título                   18254 non-null  object 
 14  Precio                

In [5]:
df_raw["Moneda"].value_counts()

Moneda
$      11158
US$     7096
Name: count, dtype: int64

In [6]:
df_raw["Marca"].value_counts()

Marca
Ford              2161
Jeep              2050
Volkswagen        2037
Chevrolet         1750
Renault           1491
Toyota            1260
Peugeot           1250
Nissan            1059
Citroën            721
BMW                672
Honda              597
Hyundai            568
Audi               529
Fiat               366
Chery              344
Kia                290
Mercedes-Benz      282
Dodge              185
BAIC               165
Suzuki              76
Porsche             67
Land Rover          65
Mitsubishi          47
Volvo               38
SsangYong           30
D.S.                27
Alfa Romeo          25
JAC                 19
Jetour              14
GWM                  9
Haval                9
Isuzu                8
Lifan                7
Lexus                7
Subaru               5
Daihatsu             4
D·S                  4
Mini                 3
Hiunday              2
KAIYI                2
DS AUTOMOBILES       2
Jaguar               2
Range Rover          1
Rrena

In [7]:
df_raw["Transmisión"].value_counts()

Transmisión
Automática               12281
Manual                    5834
Automática secuencial      123
Semiautomática               1
Name: count, dtype: int64

In [8]:
df_raw["Tipo de vendedor"].value_counts()

Tipo de vendedor
concesionaria    8509
particular       5676
tienda           4069
Name: count, dtype: int64

In [9]:
df_raw["Tipo de carrocería"].value_counts()

Tipo de carrocería
SUV    18254
Name: count, dtype: int64

In [10]:
df_raw["Tipo de combustible"].value_counts()

Tipo de combustible
Nafta            16067
Diésel            1328
Nafta/GNC          451
Híbrido/Nafta      269
Híbrido            105
GNC                 12
Eléctrico           12
Mild Hybrid         10
Name: count, dtype: int64

In [11]:
from src.data_cleaner import DataProcessor

### Pre-procesamientos del dataset

In [12]:
dp_OH = DataProcessor(df_raw)
df_OH = pd.DataFrame(dp_OH.preprocess())

df_OH.to_csv("dataset/processed/marcas_OH.csv", index=False)


In [13]:
from models.linear_regression import LinearReg

In [14]:
from src.train_val_models import*

In [15]:
resultados = []

datasets = {
    "One-hot completo": df_OH,
    # agregá más datasets si querés probar
}

for nombre, dataset in datasets.items():
    y = dataset["Precio_usd"]
    X = dataset.drop(columns=["Precio_usd"])

    X_train, X_val, y_train, y_val = split_dataset(dataset)

    # Normalizar train, obteniendo medias y stds
    X_train_norm, means, stds = NormalizeFeatures(X_train)

    # Normalizar validación usando stats del train
    X_val_norm = NormalizeWithStats(X_val, means, stds)

    res = train_val_linear_reg(
        X_train_norm.values, X_val_norm.values, y_train.values, y_val.values,
        metodo="pinv", reg="l2", l2=0.1
    )

    resultados.append({
        "nombre_dataset": nombre,
        "mse": res["mse"],
        "rmse": res["rmse"],
        "mae": res["mae"],
        "r2": res["r2"]
    })

    print(f"[{nombre}] MSE: {res['mse']:.2f} | RMSE: {res['rmse']:.2f} | R2: {res['r2']:.2f} | MAE: {res['mae']:.2f}")


[One-hot completo] MSE: 310380475957070.75 | RMSE: 17617618.34 | R2: -876546.77 | MAE: 300221.80


In [ ]:
print(X_train_norm.describe())
print(X_val_norm.describe())


### Comparar metricas de varios modelos

In [16]:
df_resultados = pd.DataFrame(resultados)
display(df_resultados.sort_values("rmse"))

,nombre_dataset,mse,rmse,mae,r2
0,One-hot completo,3.103805e+14,1.761762e+07,300221.798756,-876546.768686


In [17]:
import matplotlib.pyplot as plt

plt.scatter(y_val, y_pred_test, alpha=0.3)
plt.xlabel("Precio real")
plt.ylabel("Precio predicho")
plt.title("Predicción vs Realidad")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--")
plt.show()


NameError: name 'y_pred_test' is not defined